In [1]:
%load_ext autoreload
%autoreload 2

# World of Warcraft Log Toy

This notebook toys with data generated from World of Warcraft (WoW) logs.

4 helper classes were created:
- **Record**
  - *Parses and stores an entry of the log*
  - Each line may have differente structure so more classes deriving from this one may be necessary.
  - Each record also represent an in game event.
- **Encounter**
  - *A set of records that represents a fight agains a boss in the game*
  - Each starts with a ENCOUNTER_START event and go until an ENCOUNTER_END event is found.
- **Query**
  - *An object to help create a chain of iterators to build queries (using filters and map like functions)*
- **Predicate**
  - *A Collection of functions that will be applied to those iterators*


In [2]:
# Initialize
import pandas as pd

from wow.query import Query, Predicate
from wow.log import Record, Encounter

# file = r'wow/WoWCombatLog-012622_215632.txt'
file = r'C:\Program Files (x86)\World of Warcraft\_retail_\Logs\WoWCombatLog-012722_214646.txt'

with open(file, 'r', encoding='utf-8') as filePtr:
  raw = filePtr.readlines()


# Create Data Structures

In [3]:
# Create Record List

log = Query(enumerate(raw)).map(lambda x: Record(x)).list()

In [4]:
# Split Encounters

qLog = Query(log)
beg = qLog.filter(Predicate.isEncounterStart())
end = qLog.filter(Predicate.isEncounterEnd())

encounters = Query(zip(beg, end)).map(lambda x: Encounter(log, x[0], x[1]))

e = encounters.filter(
    lambda x: (x.timestamp_end - x.timestamp_begin).total_seconds() > 60
).list()
print(len(encounters.list()), ' => ', len(e))


0  =>  20


# Initial Analysis (Commented)

In [5]:
# Players List

# players = e[0].q.filter(
#     Predicate.isPlayerAction()
# ).map(
#     Predicate.getActorInfo()
# ).set()
# players

In [6]:
# Hostile NPCs & (some of) Their Actions

# e[0].q.filter(
#     Predicate.isCreatureAction()
# ).filter(
#     Predicate.isActorHostile()
# ).map(
#     [ Predicate.getActor(), Predicate.getEvent()]
# ).set()


In [7]:
# Orb of Torment Actions

# orb = e[0].q.filter(
#     Predicate.isActor('"Orb of Torment"')
# ).map(
#     (
#         Predicate.getAction(),
#         Predicate.getEvent(),
#         Predicate.getTarget()
#     )
# ).list()
# orb


# Detailed Analysis

In [8]:
# Define Encounter being Analysed

encounter_number = 13
e[encounter_number].md()


<style>
sb { color: steelblue }
o { color: Orange }
g { color: Green }
</style>

## <sb>"Remnant of Ner'zhul" (id: 2432)</sb>
- ENCOUNTER_START (log: 717473)
  - *2022-01-27 23:27:26.389000*  
- 79351 entries in **0:03:40.339000**
- ENCOUNTER_END (log: 796824)
  - *2022-01-27 23:31:06.728000*
    

In [9]:
# Find when orbs are spawned and when they get thrown away

from wow.fights import nerzhul
(orbSpawned, orbDisposed) = nerzhul.orbs_lifecycle(e[encounter_number])


In [10]:
# People who carried the orbs

nerzhul.orbs_carriers(orbDisposed)

,TimeStamp,ID,Player
1,2022-01-27 23:27:56.643000,Creature-0-4216-2450-10260-177117-000073549A,"""Thilokz-Azralon"""
0,2022-01-27 23:27:55.981000,Creature-0-4216-2450-10260-177117-000073549C,"""Zairoz-Azralon"""
2,2022-01-27 23:28:37.259000,Creature-0-4216-2450-10260-177117-00007354C7,"""Thilokz-Azralon"""
3,2022-01-27 23:28:43.596000,Creature-0-4216-2450-10260-177117-00007354C7,"""Spartanovix-Azralon"""
4,2022-01-27 23:28:44.977000,Creature-0-4216-2450-10260-177117-00007354C9,"""Zairoz-Azralon"""
6,2022-01-27 23:29:19.650000,Creature-0-4216-2450-10260-177117-00007354EA,"""Thilokz-Azralon"""
5,2022-01-27 23:29:13.844000,Creature-0-4216-2450-10260-177117-00007354EC,"""Djøw-Azralon"""
9,2022-01-27 23:29:58.905000,Creature-0-4216-2450-10260-177117-0000735514,"""Spartanovix-Azralon"""
7,2022-01-27 23:29:51.063000,Creature-0-4216-2450-10260-177117-0000735515,"""Thilokz-Azralon"""
8,2022-01-27 23:29:54.010000,Creature-0-4216-2450-10260-177117-0000735517,"""Flintti-Azralon"""


In [11]:
# Orbs life span and who threw it away (for good)

nerzhul.orbs_overview(orbSpawned, orbDisposed)


,ID,TimeStamp Spawned,TimeStamp Disposed,Time Elapsed,Player
0,Creature-0-4216-2450-10260-177117-000073549A,2022-01-27 23:27:37.306000,2022-01-27 23:27:56.643000,0 days 00:00:19.337000,"""Thilokz-Azralon"""
1,Creature-0-4216-2450-10260-177117-000073549C,2022-01-27 23:27:38.817000,2022-01-27 23:27:55.981000,0 days 00:00:17.164000,"""Zairoz-Azralon"""
2,Creature-0-4216-2450-10260-177117-00007354C7,2022-01-27 23:28:22.275000,2022-01-27 23:28:43.596000,0 days 00:00:21.321000,"""Spartanovix-Azralon"""
3,Creature-0-4216-2450-10260-177117-00007354C9,2022-01-27 23:28:23.775000,2022-01-27 23:28:44.977000,0 days 00:00:21.202000,"""Zairoz-Azralon"""
4,Creature-0-4216-2450-10260-177117-00007354EA,2022-01-27 23:28:57.487000,2022-01-27 23:29:19.650000,0 days 00:00:22.163000,"""Thilokz-Azralon"""
5,Creature-0-4216-2450-10260-177117-00007354EC,2022-01-27 23:28:59.025000,2022-01-27 23:29:13.844000,0 days 00:00:14.819000,"""Djøw-Azralon"""
6,Creature-0-4216-2450-10260-177117-0000735514,2022-01-27 23:29:38.807000,2022-01-27 23:29:58.905000,0 days 00:00:20.098000,"""Spartanovix-Azralon"""
7,Creature-0-4216-2450-10260-177117-0000735515,2022-01-27 23:29:40.281000,2022-01-27 23:29:51.063000,0 days 00:00:10.782000,"""Thilokz-Azralon"""
8,Creature-0-4216-2450-10260-177117-0000735517,2022-01-27 23:29:41.790000,2022-01-27 23:29:54.010000,0 days 00:00:12.220000,"""Flintti-Azralon"""
9,Creature-0-4216-2450-10260-177117-0000735538,2022-01-27 23:30:15.361000,2022-01-27 23:30:40.347000,0 days 00:00:24.986000,"""Spartanovix-Azralon"""
